In [11]:
import sys
import os
import datetime

# Add the directory containing 'schema' to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import uuid
from schema.post import Post, ReactionType
import random
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

In [2]:
parser = PydanticOutputParser(pydantic_object=Post)
job_title = "Machine Learning Engineer"
company_name = "CVS Health"
user_id = "1234"
format_instructions = parser.get_format_instructions()
role = "recruiter"
job_id = "1234"

In [3]:
job_description = """
**Company Description  
            CVS Health is committed to delivering human-centric healthcare through innovation and empathy. The company emphasizes  Heart At Work Behaviors™ *, fostering a culture of empowerment, collaboration, and purpose-driven solutions.  

            ---

            **Position Details**  
            **Role Summary:**  
            Design, develop, and deploy machine learning solutions to enhance healthcare services. Collaborate with cross-functional teams to translate data insights into production-ready tools.  

            **Key Responsibilities:**  
            - Partner with data scientists, developers, and product managers to operationalize ML services.  
            - Build and optimize ML models/frameworks for user value.  
            - Develop data pipelines and leverage cloud warehouses (BigQuery, Snowflake, Redshift).  
            - Use modern software stacks (Python, Java, SQL, PySpark) for ML lifecycle management.  
            - Lead product performance improvements and stakeholder communication.  

            ---

            **Qualifications**  
            - **Technical Skills:**  
              - 2+ years in software development (Python, Java, or open-source contributions).  
              - 1+ years with ML libraries (Scikit-learn, PyTorch, etc.) and algorithms (clustering, decision trees).  
              - 1+ years in analytical pipelines (SQL, PySpark) and cloud data tools.  
              - 1+ years with Docker/Kubernetes.  
            - **Soft Skills:**  
              - Stakeholder management and independent project delivery.  

            ---

            **Education**  
            - **Required:** Bachelor’s degree in Computer Science, Data Engineering, ML, or related field.  
            - **Preferred:** Master’s degree.  

            ---

            **Compensation & Benefits**  
            - **Weekly Hours:** 40  
            - **Pay Range:** $86,520 - $173,040 annually (base salary + bonus/commission eligibility).  
            - **Benefits:**  
              - Medical, dental, vision insurance.  
              - 401(k), Employee Stock Purchase Plan.  
              - Life/disability insurance, wellness programs, education assistance.  
              - PTO, paid holidays, CVS store discounts.  

            ---

            **Application Information**  
            - **Application Window Closes:** March 31, 2025.  
            - **Note:** CVS Health considers applicants with arrest/conviction records per federal, state, and local laws.  

            **Simplify Your Application:** Focus on aligning technical expertise (ML, cloud tools) and collaborative experience in healthcare or related fields. Highlight open-source contributions and stakeholder management skills.  

            ---  
            *Structured for clarity, brevity, and emphasis on key requirements and cultural fit.*
"""

In [4]:
# Create the prompt template
from langchain.prompts import PromptTemplate
post_template = PromptTemplate.from_template(
    """Create a LinkedIn recruiter post for a {job_title} position following this exact structure:

🚀 We're Hiring: {job_title}!
✨ About the Role:
- {{Insert 1-sentence company introduction}}
- {{Share core team mission}}

🔍 Key Responsibilities:
{{
- Extract 3-5 main responsibilities from job description
- Focus on impact rather than tasks
}}

✅ Ideal Candidate:
{{
- List 3-5 key requirements from job description
- Include both hard and soft skills
}}

🌟 Why Join Us?
{{{{
- Highlight 2-3 unique benefits/opportunities
- Keep it employee-centric
}}}}

📩 How to Apply:
- Include clear call-to-action
- Add relevant hashtags (3-5)

Job Description Context: {job_description}

Maintain these formatting rules:
- Use emojis as section headers
- Keep paragraphs under 3 lines
- Use clean line breaks between sections
- Avoid markdown formatting
- Maintain corporate tone with personality"""
)

In [5]:
import os
from dotenv import load_dotenv

class Config:
    def __init__(self):
        load_dotenv()  # Load .env variables

        self.OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
        self.DB_NAME = os.getenv("DB_NAME")
        self.DB_CREDENTIALS_PATH = os.getenv("DB_CREDENTIALS_PATH")
        self.MAX_OPEN_AI_REQUEST_PER_MIN = 20
        self.MAX_OPEN_AI_REQUEST_PER_DAY = 200
        
# Create a single config instance
config = Config()

In [ ]:
LLM_CONFIGS = {
    'user-recruiter-generation': {
        "model_name": "meta-llama/llama-3.3-70b-instruct:free",
        "params": {
            "temperature":1.2,
            "max_tokens": 50000,
            "top_p":1.0,
            "frequency_penalty": 1.0,
            "presence_penalty": 0.0,
            "request_timeout": 60
        }
    },
    
}

In [7]:
def get_open_router_llm(config_key):
    open_router_base_url = "https://openrouter.ai/api/v1"
    llm_params = LLM_CONFIGS[config_key]
    if not config:
        raise ValueError(f"Configuration for {config_key} not found.")
    model_name = llm_params["model_name"]
    model_params = llm_params["params"]

    if not model_name:
        raise ValueError(f"Model name not found for {config_key}.")
    
    llm = ChatOpenAI(
        model_name = llm_params["model_name"],
        openai_api_base = open_router_base_url,
        openai_api_key = config.OPENAI_API_KEY,
        **model_params
    )
    print("\nLLm Used: \n", llm)
    return llm

In [8]:
llm = get_open_router_llm("user-recruiter-generation")
chain = post_template | llm 


LLm Used: 
 client=<openai.resources.chat.completions.completions.Completions object at 0x118ea1750> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x119007450> root_client=<openai.OpenAI object at 0x118ef1b50> root_async_client=<openai.AsyncOpenAI object at 0x118fe8590> model_name='meta-llama/llama-3.3-70b-instruct:free' temperature=1.2 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://openrouter.ai/api/v1' request_timeout=60.0 presence_penalty=0.0 frequency_penalty=1.0 top_p=1.0 max_tokens=50000


In [10]:
post_content = chain.invoke({
    "job_title": job_title,
    "job_description": job_description
})

print(post_content.content)


🚀 We're Hiring: Machine Learning Engineer!
✨ About the Role:
CVS Health delivers human-centric healthcare through innovation and empathy. 
Our team's mission is to enhance healthcare services through machine learning solutions.

🔍 Key Responsibilities:
- Collaborate with cross-functional teams to operationalize ML services
- Develop production-ready tools that drive user value
- Optimize ML models for better performance

✅ Ideal Candidate:
- 2+ years of software development experience with Python, Java, or open-source contributions
- Experience with machine learning libraries and algorithms
- Strong stakeholder management and independent project delivery skills

🌟 Why Join Us?
Unique benefits include professional growth opportunities, comprehensive health insurance, and a culture of empowerment 
Collaborative work environment with a purpose-driven team
Opportunities for open-source contributions and innovation

📩 How to Apply:
Apply now at our website or email your resume to [insert em

In [19]:
# Create validated Post object
valid_post = Post(
    job_id=datetime.datetime.now().strftime("%Y%m%d%H%M%S"),
    timestamp=int(datetime.datetime.now().timestamp()),
    author="1234",
    content=post_content.content,
    ttl=0,  # Will be auto-calculated
    views=random.randint(0, 1000),
    comments=[],
    likes=[{"user_id": datetime.datetime.now().strftime("%Y%m%d%H%M%S"), "reaction_type": random.choice(list(ReactionType)), "created_at": int(datetime.datetime.now().timestamp())} for _ in range(10)],
    repost={"timestamp": "", "count": 0},
)

In [23]:
valid_post.ttl

1747743492